In [4]:
import os

# The below line was causing Java gateway Process exited before sending its port number error
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars $PKG_HOME/snowflake-jdbc-3.6.10.jar,$PKG_HOME/spark-snowflake_2.11-2.4.4-spark_2.2.jar,$PKG_HOME/hadoop-aws-2.7.3.jar,$PKG_HOME/aws-java-sdk-1.7.4.jar'
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_161.jdk/Contents/Home'

In [6]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *



spark_conf = SparkConf().setMaster('local').setAppName('OfferUp_Spark_Snowflake')
sc = SparkContext.getOrCreate()
# spark = SparkSession(sc)

In [7]:
#
# Some constants
#
aws_profile = "prasad_dev"
aws_region = "us-east-1"
s3_bucket = "ou-staging"

# 
# Reading environment variables from aws credential file 
#
import configparser

config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))

access_id = config.get(aws_profile, "aws_access_key_id") 
access_key = config.get(aws_profile, "aws_secret_access_key") 


# You might need to set these
spark._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", access_id)
spark._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", access_key)

# Set options below
# For accounts in US West, sfURL format is <account_name>.snowflakecomputing.com (i.e. no region ID)
# For all other regions, sfURL format is <account_name>.<region_id>.snowflakecomputing.com
sfOptions = {
  "sfURL" : "offerup.snowflakecomputing.com",
  "sfAccount" : "offerup",
  "sfUser" : "",
  "sfPassword" : "",
  "sfDatabase" : "ou_users",
  "sfSchema" : "prasad",
  "sfWarehouse" : "analytics_wh",
}

SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query",  "select * from DATA_SPIKE_POSTERS_AND_ITEMS") \
  .load()

df.show(10)

sc.stop()

NoSectionError: No section: 'prasad_dev'

Exception: Java gateway process exited before sending its port number

In [3]:
import random
spark_conf = SparkConf().setMaster('local').setAppName('Pi')
sc = SparkContext.getOrCreate()
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.14129616
